In [1]:
import scipy.io
import tensorflow as tf
import numpy as np
from tensorflow import  keras
from sklearn.preprocessing import MinMaxScaler,StandardScaler,scale
from skimage.measure import block_reduce
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout

In [2]:
def load_data():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["data"].transpose()  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['data'].transpose() # (262134, 270) 
    myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['data'].transpose() #(262134, 315)
    return als,normal,myopathie

In [3]:

def scale_data_minmax(data):
    scaler = MinMaxScaler()
    normalized_data =scaler.fit_transform(data)
    return normalized_data

In [4]:
def scale_data_standard(data):
    scaled_data = scale(data,axis=1)
    return scaled_data

In [5]:
def gen_tensors_list(data,dim):
    dataset  = list()
    for i in range (len(data)):
    arg = tf.convert_to_tensor(data[i], dtype=tf.float32)
    arg = tf.reshape(arg,dim)
    dataset.append(arg)
    return dataset

In [6]:
def windowing_fun(data,window_size=1000,overlap=100):
    windowed_data = [data[j][i : i + window_size] for j in range(0,len(data)) for i in range(0, len(data[j]), window_size-overlap)]
    final_data = [windowed_data[i] for i in range(len(windowed_data)) if len(windowed_data[i]) == window_size ]
    return final_data

In [7]:
def load_file_names():
    als = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/ALS.mat',)["files"]  #(262134, 321)
    normal = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/normal.mat')['files'] # (262134, 270) 
    myopathie = scipy.io.loadmat('C:/Users/lenovo/OneDrive/Documents/drive-download-20230214T190624Z-001/myopathie.mat')['files'] #(262134, 315)
    return als,normal,myopathie

In [16]:
def generate_dataframe(data,files):
    df = pd.DataFrame([], columns=['type_signal',"num_personne","muscle","num_enregistrement","signal"])
    windowed_data = [{"signal": data[j],"type_signal":files[0][j][0][5],"num_personne": files[0][j][0][6:8],"muscle": files[0][j][0][8:10],"num_enregistrement":files[0][j][0][10:12] } for j in range(0,len(data)) ]
    final_data = pd.DataFrame(windowed_data)
    df = pd.concat([df,final_data], ignore_index=True)
    return df

In [17]:
als, normal , myopathie = load_data()

In [18]:
alsfiles,normalfiles,myopathiefiles = load_file_names()

In [19]:
df = generate_dataframe(np.vstack((normal,als,myopathie)),np.concatenate((normalfiles,alsfiles,myopathiefiles),axis=1))

In [20]:
def f(row):
  return {"C":0,"A":1,"M":2}[row["type_signal"]]

df["num_class"] =df.apply(f, axis=1) 

In [21]:
df = df.query("muscle=='BB'")

In [205]:
X = df['signal'].tolist()
Y = df['num_class'].tolist()

In [206]:
X=X[170:]
Y=Y[170:]

In [207]:
del df

In [208]:
data = [*X]
#data, np.shape(data)

In [209]:
del  als,normal,myopathie,alsfiles,normalfiles,myopathiefiles 

In [210]:
data = scale_data_standard(data)

In [211]:
last_index_normal = len(Y) - 1 - Y[::-1].index(0)

In [212]:
last_index_als = len(Y) - 1 - Y[::-1].index(1)

In [213]:
last_index_myopathie = len(Y) - 1 - Y[::-1].index(2)

In [214]:
windowed_normal = len(windowing_fun(X[:last_index_normal+1],10000))
windowed_als = len(windowing_fun(X[last_index_normal+1:last_index_als+1],10000))
windowed_myopathie = len(windowing_fun(X[last_index_als+1:],10000))

In [215]:
Y = [0]*(windowed_normal)+[1]*(windowed_als)+[2]*(windowed_myopathie)

In [216]:
Y  = Y*1

In [217]:
X = windowing_fun(data,10000)

In [218]:
del data

In [219]:
X = scale_data_standard(X)

In [220]:
tensors_list = gen_tensors_list(X,(1,10000))

In [228]:
X_train, X_test, y_train, y_test = train_test_split( tensors_list, Y, test_size=0.04,random_state=20)
np.shape(X_train), np.shape(X_test)

((7612, 1, 10000), (318, 1, 10000))

In [229]:
model = Sequential([
    #Conv2D(#of filters, filters size, activation function)
    #in keras for the first layer we always need to mention the input shape
    Conv1D(64, 3, activation = 'relu', input_shape = (1,10000), padding='same'), 
    
    #MaxPooling2D(filter size stride and pad can also be added)
    MaxPooling1D(2, strides = 2, padding='same'),
    Dropout(0.2),
    
    #LAYER2
    Conv1D(32, 3, activation = 'relu', padding='same'),
    MaxPooling1D(2, strides = 2, padding='same'),
    Dropout(0.4),
    
    #FLATTEN
    Flatten(),
    
    #FC Layers:
    
    #LAYER3
    #Dense : fully connected  Dense(#neurons, activation function )
    Dense(64, activation = 'relu'),
    Dropout(0.6),
    
    #LAYER4
    Dense(3, activation = 'softmax'),
    
    
])


In [230]:
logdir = "drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir=logdir),tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',patience =60 ,min_delta=1e-3)]


In [231]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [232]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_text = np.asarray(X_test)
y_test = np.asarray(y_test)

In [233]:
model.fit(X_train, y_train, epochs =50, batch_size = 128)

Epoch 1/50
60/60 [==============================] - 12s 163ms/step - loss: 1.2741 - accuracy: 0.3318
Epoch 2/50
60/60 [==============================] - 10s 159ms/step - loss: 1.1378 - accuracy: 0.3492
Epoch 3/50
60/60 [==============================] - 9s 154ms/step - loss: 1.1013 - accuracy: 0.3739
Epoch 4/50
60/60 [==============================] - 9s 158ms/step - loss: 1.0973 - accuracy: 0.3760
Epoch 5/50
60/60 [==============================] - 10s 161ms/step - loss: 1.0919 - accuracy: 0.3802
Epoch 6/50
60/60 [==============================] - 10s 162ms/step - loss: 1.0833 - accuracy: 0.3916
Epoch 7/50
60/60 [==============================] - 10s 165ms/step - loss: 1.0748 - accuracy: 0.4045
Epoch 8/50
60/60 [==============================] - 10s 160ms/step - loss: 1.0603 - accuracy: 0.4263
Epoch 9/50
60/60 [==============================] - 9s 157ms/step - loss: 1.0519 - accuracy: 0.4364
Epoch 10/50
60/60 [==============================] - 9s 156ms/step - loss: 1.0461 - accuracy: 

In [234]:
score = model.evaluate(np.asarray(X_test),np.asarray (y_test))

10/10 [==============================] - 1s 8ms/step - loss: 1.7098 - accuracy: 0.3333


In [198]:
%reload_ext tensorboard

In [199]:
%tensorboard --logdir drive/MyDrive/PFE_TENSORBOARD_DATA/augmentation0/CNN1D/logs/scalars/

Reusing TensorBoard on port 6006 (pid 12412), started 4:22:36 ago. (Use '!kill 12412' to kill it.)

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/x_train.npy', 'wb') as f:
    np.save(f, np.asarray(X_train))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/x_test.npy', 'wb') as f:
    np.save(f, np.asarray(X_test))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/y_train.npy', 'wb') as f:
    np.save(f, np.asarray(y_train))

In [ ]:
with open('/content/drive/MyDrive/SAVED_DATA/healthy_malade/augmentation4/y_test.npy', 'wb') as f:
    np.save(f, np.asarray(y_test))